In [1]:
import itertools
import pandas as pd

In [2]:
import networkx as nx
G = nx.Graph()

In [3]:
df_train = pd.read_json('train.json')
df_train_seperated = df_train
df_train_seperated['coauth'] = ""

df_test = pd.read_json('test.json')
df_test_auth = df_test['coauthors']

for index, row in enumerate(df_train.values):
    df_train_seperated['authors'][index] = [item for item in row[0] if item < 100 ]
    df_train_seperated['coauth'][index] = [item for item in row[0] if item >= 100 ]

/var/folders/bs/g0pg3lcs4lxfqrsc30blzc040000gn/T/ipykernel_5596/1277698713.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_seperated['authors'][index] = [item for item in row[0] if item < 100 ]
/var/folders/bs/g0pg3lcs4lxfqrsc30blzc040000gn/T/ipykernel_5596/1277698713.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_seperated['coauth'][index] = [item for item in row[0] if item >= 100 ]


In [4]:
df_train_seperated.head()

,authors,year,abstract,venue,title,coauth
0,"[42, 36]",9,"[2455, 1858, 2335, 1543, 1800, 1860, 2000, 286...",20,"[41, 1550, 1563, 1594, 1544, 1919, 1644, 37, 1...",[13720]
1,[45],15,"[40, 1542, 1691, 2449, 1535, 3616, 2206, 1904,...",2,"[1731, 47, 11, 57, 4624, 1525, 1535, 47, 11, 3...","[1359, 15881]"
2,[],17,"[40, 1542, 1691, 2449, 1535, 2610, 1543, 1535,...",,"[2085, 1719, 1846, 1745, 2243, 1553, 1606, 159...","[19166, 17763]"
3,[97],10,"[46, 1624, 1547, 56, 1687, 1644, 6, 7, 3386, 1...",4,"[40, 1733, 1735, 1540, 1655, 46, 1624, 1547, 5...",[]
4,[2],10,"[37, 3709, 3836, 1586, 2151, 1727, 3021, 1860,...",9,"[38, 1592, 2088, 1543, 1574, 1727, 1597, 1813,...",[19617]


In [5]:
#to create dataframe of cross products of relations
li = []
for index,row in enumerate(df_train_seperated.values):
    a = row[0]
    b = row[5]
    li.extend(list(itertools.product(a,b)))

In [6]:
df = pd.DataFrame(li)
df = df.rename(columns={0: 'Author1', 1: 'Author2'})

In [7]:
df.head(10)

,Author1,Author2
0,42,13720
1,36,13720
2,45,1359
3,45,15881
4,2,19617
5,44,9641
6,44,5623
7,2,9641
8,2,5623
9,97,3241


In [8]:
auth_nodes = []
for i,li in enumerate(df_test_auth):
    auth_nodes.extend(li)

auth_rel = []
for i,li in enumerate(df_test_auth):
    auth_rel.extend(list(itertools.combinations(li,2)))
    
auth_nodes = list(set(auth_nodes))

#add nodes for author < 100
p_auth = df['Author1'].tolist()

#add nodes for author >= 100
ci_auth = df['Author2'].tolist()

d = []
d.extend(p_auth)
d.extend(ci_auth)
d.extend(auth_nodes)

nodes = set(d)
nodes = list(nodes)
test_auth_edges = auth_rel

In [9]:
#adding nodes to the graph
G.add_nodes_from(nodes)
G.number_of_nodes()

7804

In [10]:
#adding edges to the graph
edge = list(zip(p_auth, ci_auth))
edge.extend(auth_rel)

edge = set(edge)
edge = list(edge)
G.add_edges_from(edge)
G.number_of_edges()

20201

In [11]:
#Testing the graph
nx.shortest_path_length(G, source=19517, target=42)

1

In [12]:
%%time
#Testing
print(nx.shortest_path(G, source=2, target=21))
li = nx.shortest_path(G, source=2, target=21)
li[1:-1]

[2, 1333, 76, 11960, 21]
CPU times: user 300 µs, sys: 15 µs, total: 315 µs
Wall time: 312 µs


[1333, 76, 11960]

# Finding nearest authors in k hops

In [13]:
def nearest_authors(li):
    li = list(li)
    di = {}
    combined = []
    mod_li = []
    for index, target in enumerate(li):
        if target in list(G.nodes):
            mod_li.append(target)
    
    if len(mod_li) == 0:
        return []

    for index,i in enumerate(mod_li):
        match = []
        for ds in range(0,3):
            match.extend(list(nx.descendants_at_distance(G, i, ds)))
        
        match = [item for item in match if item < 100]
        combined.append(match)
        
    return list(set.intersection(*map(set,combined)))

In [14]:
%%time
#testing
li = [16336, 1762, 4357, 12564]
nearest_authors(li)

CPU times: user 1.04 ms, sys: 7 µs, total: 1.05 ms
Wall time: 1.05 ms


[92]

# Making search base

In [15]:
df_test.head()

,identifier,coauthors,year,abstract,venue,title
0,0,"[16336, 1762, 4357, 12564]",19,"[37, 1662, 3207, 10, 33, 2037, 1738, 1642, 155...",223,"[3207, 24, 1798, 1738, 37, 2375, 1568, 11, 53,..."
1,1,"[21189, 14088]",19,"[1731, 2130, 3674, 1705, 1656, 3077, 1546, 367...",223,"[40, 1560, 1536, 1544, 1609, 1705, 1658, 1543,..."
2,2,"[3625, 1198, 19889, 794, 2749, 7801]",19,"[1551, 1728, 3920, 1542, 1535, 1656, 1543, 153...",7,"[47, 1574, 1729, 1641, 11, 37, 2533, 2015, 47,..."
3,3,"[19810, 15173, 5876, 111]",19,"[51, 1535, 2115, 1543, 1811, 1700, 1657, 1684,...",21,"[1770, 53, 2054, 1549, 1529, 1723, 2796, 1547,..."
4,4,"[10932, 7668, 11907, 19601, 15307, 10492, 1049...",19,"[1775, 1746, 1842, 1525, 33, 2551, 1882, 1542,...",,"[18, 1924, 23, 1544, 3927, 2686, 1543, 1535, 1..."


In [16]:
%%time
df_test["probable_auth"] = df_test['coauthors'].apply(lambda x : nearest_authors(x) )


CPU times: user 566 ms, sys: 1.51 ms, total: 568 ms
Wall time: 567 ms


In [17]:
df_test.head(5)

,identifier,coauthors,year,abstract,venue,title,probable_auth
0,0,"[16336, 1762, 4357, 12564]",19,"[37, 1662, 3207, 10, 33, 2037, 1738, 1642, 155...",223,"[3207, 24, 1798, 1738, 37, 2375, 1568, 11, 53,...",[92]
1,1,"[21189, 14088]",19,"[1731, 2130, 3674, 1705, 1656, 3077, 1546, 367...",223,"[40, 1560, 1536, 1544, 1609, 1705, 1658, 1543,...","[2, 6, 8, 11, 17, 22, 26, 27, 28, 31, 32, 42, ..."
2,2,"[3625, 1198, 19889, 794, 2749, 7801]",19,"[1551, 1728, 3920, 1542, 1535, 1656, 1543, 153...",7,"[47, 1574, 1729, 1641, 11, 37, 2533, 2015, 47,...","[79, 50, 51, 55, 31]"
3,3,"[19810, 15173, 5876, 111]",19,"[51, 1535, 2115, 1543, 1811, 1700, 1657, 1684,...",21,"[1770, 53, 2054, 1549, 1529, 1723, 2796, 1547,...",[23]
4,4,"[10932, 7668, 11907, 19601, 15307, 10492, 1049...",19,"[1775, 1746, 1842, 1525, 33, 2551, 1882, 1542,...",,"[18, 1924, 23, 1544, 3927, 2686, 1543, 1535, 1...","[6, 8, 14, 15, 17, 23, 26, 28, 29, 32, 35, 37,..."


In [18]:
#getting csv file
df_test.to_csv('test_2_hop.csv')